In [2]:
#Import Libraries
import pandas as pd
import numpy as np
import string
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
from google.colab import files
#Upload IMDB_Dataset.csv file from local system to remote colab location
files.upload()

Saving IMDB_Dataset.csv to IMDB_Dataset.csv


In [6]:
#Create dataframe and store the data from IMDB_Dataset.csv
data = pd.DataFrame()
data = pd.read_csv('IMDB_Dataset.csv', encoding='utf-8')
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [7]:
#create empty list
review_data_list = list()

indv_lines = data['review'].values.tolist()
for line in indv_lines:

	#create word tokens as well as remove puntuation in one go
	rem_tok_punc = RegexpTokenizer(r'\w+')

	tokens = rem_tok_punc.tokenize(line)


	#convert the words to lower case
	words = [w.lower() for w in tokens]

	#Invoke all the english stopwords
	stop_word_list = set(stopwords.words('english'))

	#Remove stop words
	words = [w for w in words if not w in stop_word_list]

	#Append words in the review_data_list list.
	review_data_list.append(words)
len(review_data_list)

50000

In [8]:
#Train a Word2Vec model using Gensim
import gensim
Embedding_Dim = 100
#train word2vec model
model = gensim.models.Word2Vec(sentences = review_data_list, size = Embedding_Dim, workers = 4, min_count = 1)
#Vocabulary size
words = list(model.wv.vocab)
print('Here is the Vocabulary Size.. %d' % len(words))

Here is the Vocabulary Size.. 101791


In [12]:
#Finding similar words
model.wv.most_similar('amazing')


/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('incredible', 0.8465273976325989),
 ('fantastic', 0.8136316537857056),
 ('wonderful', 0.7753651142120361),
 ('outstanding', 0.7413761615753174),
 ('awesome', 0.7389760613441467),
 ('brilliant', 0.7296733856201172),
 ('superb', 0.7106024026870728),
 ('fabulous', 0.7044234275817871),
 ('excellent', 0.7033034563064575),
 ('stunning', 0.7002643346786499)]

In [10]:
model.wv.most_similar('awful')

/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('terrible', 0.8716390132904053),
 ('horrible', 0.8612148761749268),
 ('dreadful', 0.8571207523345947),
 ('atrocious', 0.8318002820014954),
 ('lousy', 0.8174184560775757),
 ('horrid', 0.8009194135665894),
 ('sucks', 0.7898591160774231),
 ('abysmal', 0.7734290361404419),
 ('pathetic', 0.7643465995788574),
 ('horrendous', 0.7613471746444702)]

In [11]:
#Performing some mathematics on word vectors queen + man - woman = ?
model.wv.most_similar_cosmul(positive=['queen','man'], negative=['woman'])

[('iii', 0.9031829833984375),
 ('legend', 0.8847224712371826),
 ('aka', 0.8751254081726074),
 ('savage', 0.874505877494812),
 ('pfalz', 0.8636574149131775),
 ('vs', 0.8631209135055542),
 ('ii', 0.8616292476654053),
 ('samurai', 0.8609600067138672),
 ('hunter', 0.8550093173980713),
 ('warrior', 0.8518388271331787)]

In [14]:
#Finding the odd word out from the list of words given
print(model.wv.doesnt_match("man woman car".split()))

car


/usr/local/lib/python3.6/dist-packages/gensim/models/keyedvectors.py:895: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  vectors = vstack(self.word_vec(word, use_norm=True) for word in used_words).astype(REAL)
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


In [16]:
# Importing bokeh libraries for showing how words of similar context are grouped together
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook

#Defining the chart
output_notebook()
plot_chart = bp.figure(plot_width=700, plot_height=600, title="A map/plot of 5000 word vectors",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

#Extracting the list of word vectors, limiting to 5000, each is of 200 dimensions
word_vectors = [model[w] for w in list(model.wv.vocab.keys())[:5000]]

# Reducing dimensionality by converting the vectors to 2d vectors
from sklearn.manifold import TSNE
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_w2v = tsne_model.fit_transform(word_vectors)

# Storing data in a dataframe
tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = list(model.wv.vocab.keys())[:5000]

# Corresponding word appears when you hover on the data point.
plot_chart.scatter(x='x', y='y', source=tsne_df)
hover = plot_chart.select(dict(type=HoverTool))
hover.tooltips={"word": "@words"}
show(plot_chart)


Loading BokehJS ...

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  if sys.path[0] == '':


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 5000 samples in 0.016s...
[t-SNE] Computed neighbors for 5000 samples in 5.854s...
[t-SNE] Computed conditional probabilities for sample 1000 / 5000
[t-SNE] Computed conditional probabilities for sample 2000 / 5000
[t-SNE] Computed conditional probabilities for sample 3000 / 5000
[t-SNE] Computed conditional probabilities for sample 4000 / 5000
[t-SNE] Computed conditional probabilities for sample 5000 / 5000
[t-SNE] Mean sigma: 0.275955
[t-SNE] KL divergence after 250 iterations with early exaggeration: 83.059464
[t-SNE] KL divergence after 1000 iterations: 2.246158


In [0]:
#Save word embedding model
model_file = 'imdb_word2vec_embedding.txt'
model.wv.save_word2vec_format(filename, binary=False)